In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv(r'/Users/allen/Documents/GitHub/ML7331-Lab1/airline_merged.csv',low_memory=False)

In [3]:
pd.set_option('display.max_columns', None)

In [4]:
#removing unwanted column from previous export
df.pop('Unnamed: 0')

0                0
1                1
2                2
3                3
4                4
            ...   
5819074    5819074
5819075    5819075
5819076    5819076
5819077    5819077
5819078    5819078
Name: Unnamed: 0, Length: 5819079, dtype: int64

In [5]:
df.head()

,year,month,day,day_of_week,airline_cd,airline_name,flight_number,tail_number,origin_airport,airport_origin,city_origin,state_origin,country_origin,latitude_origin,longitude_origin,destination_airport,airport_destination,city_destination,state_destination,country_destination,latitude_destination,longitude_destination,scheduled_departure,departure_time,departure_delay,taxi_out,wheels_off,scheduled_time,elapsed_time,air_time,distance,wheels_on,taxi_in,scheduled_arrival,arrival_time,arrival_delay,diverted,cancelled,cancellation_reason,air_system_delay,security_delay,airline_delay,late_aircraft_delay,weather_delay
0,2015,1,1,4,AS,Alaska Airlines Inc.,98,N407AS,ANC,Ted Stevens Anchorage International Airport,Anchorage,AK,USA,61.17432,-149.99619,SEA,Seattle-Tacoma International Airport,Seattle,WA,USA,47.44898,-122.30931,5,2354.0,-11.0,21.0,15.0,205.0,194.0,169.0,1448,404.0,4.0,430,408.0,-22.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
1,2015,1,1,4,AA,American Airlines Inc.,2336,N3KUAA,LAX,Los Angeles International Airport,Los Angeles,CA,USA,33.94254,-118.40807,PBI,Palm Beach International Airport,West Palm Beach,FL,USA,26.68316,-80.09559,10,2.0,-8.0,12.0,14.0,280.0,279.0,263.0,2330,737.0,4.0,750,741.0,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
2,2015,1,1,4,US,US Airways Inc.,840,N171US,SFO,San Francisco International Airport,San Francisco,CA,USA,37.61900,-122.37484,CLT,Charlotte Douglas International Airport,Charlotte,NC,USA,35.21401,-80.94313,20,18.0,-2.0,16.0,34.0,286.0,293.0,266.0,2296,800.0,11.0,806,811.0,5.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
3,2015,1,1,4,AA,American Airlines Inc.,258,N3HYAA,LAX,Los Angeles International Airport,Los Angeles,CA,USA,33.94254,-118.40807,MIA,Miami International Airport,Miami,FL,USA,25.79325,-80.29056,20,15.0,-5.0,15.0,30.0,285.0,281.0,258.0,2342,748.0,8.0,805,756.0,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,2015,1,1,4,AS,Alaska Airlines Inc.,135,N527AS,SEA,Seattle-Tacoma International Airport,Seattle,WA,USA,47.44898,-122.30931,ANC,Ted Stevens Anchorage International Airport,Anchorage,AK,USA,61.17432,-149.99619,25,24.0,-1.0,11.0,35.0,235.0,215.0,199.0,1448,254.0,5.0,320,259.0,-21.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
#selecting states in southeast
airports = ['ATL','DFW','ORD','LAX','DEN']
#airports = ['DFW']

In [7]:
df_air = df.loc[df['origin_airport'].isin(airports)]

In [8]:
df_air.describe()

,year,month,day,day_of_week,flight_number,latitude_origin,longitude_origin,latitude_destination,longitude_destination,scheduled_departure,departure_time,departure_delay,taxi_out,wheels_off,scheduled_time,elapsed_time,air_time,distance,wheels_on,taxi_in,scheduled_arrival,arrival_time,arrival_delay,diverted,cancelled,air_system_delay,security_delay,airline_delay,late_aircraft_delay,weather_delay
count,1262999.0,1.262999e+06,1.262999e+06,1.262999e+06,1.262999e+06,1.262999e+06,1.262999e+06,1.260978e+06,1.260978e+06,1.262999e+06,1.242452e+06,1.242452e+06,1.241645e+06,1.241645e+06,1.262997e+06,1.238112e+06,1.238112e+06,1.262999e+06,1.240709e+06,1.240709e+06,1.262999e+06,1.240709e+06,1.238112e+06,1.262999e+06,1.262999e+06,257215.000000,257215.000000,257215.000000,257215.000000,257215.000000
mean,2015.0,6.213842e+00,1.569226e+01,3.921471e+00,2.285561e+03,3.639861e+01,-9.598541e+01,3.672163e+01,-9.420490e+01,1.406318e+03,1.411853e+03,1.142344e+01,1.755801e+01,1.432995e+03,1.392324e+02,1.343131e+02,1.103290e+02,8.153202e+02,1.534275e+03,6.438634e+00,1.559219e+03,1.533658e+03,6.076924e+00,2.566114e-03,1.713857e-02,12.022289,0.053823,19.442008,22.024761,4.342845
std,0.0,3.375813e+00,8.773835e+00,1.995448e+00,1.749664e+03,3.775494e+00,1.186905e+01,5.577654e+00,1.666615e+01,4.784735e+02,4.921992e+02,3.621018e+01,8.850682e+00,4.965999e+02,7.117940e+01,7.067034e+01,6.897915e+01,5.857315e+02,5.319540e+02,4.948108e+00,5.133976e+02,5.393105e+02,3.859071e+01,5.059181e-02,1.297877e-01,25.042409,1.693780,39.512372,41.056916,24.214909
min,2015.0,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,3.289595e+01,-1.184081e+02,1.770189e+01,-1.593390e+02,1.000000e+00,1.000000e+00,-5.200000e+01,1.000000e+00,1.000000e+00,3.300000e+01,2.400000e+01,1.100000e+01,6.700000e+01,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,-8.000000e+01,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2015.0,3.000000e+00,8.000000e+00,2.000000e+00,9.200000e+02,3.364044e+01,-1.046670e+02,3.289595e+01,-1.046670e+02,1.010000e+03,1.013000e+03,-4.000000e+00,1.200000e+01,1.029000e+03,8.700000e+01,8.300000e+01,6.000000e+01,3.910000e+02,1.146000e+03,4.000000e+00,1.201000e+03,1.148000e+03,-1.300000e+01,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000
50%,2015.0,6.000000e+00,1.600000e+01,4.000000e+00,1.782000e+03,3.394254e+01,-8.790446e+01,3.750517e+01,-9.007589e+01,1.400000e+03,1.410000e+03,-1.000000e+00,1.600000e+01,1.425000e+03,1.220000e+02,1.180000e+02,9.300000e+01,6.660000e+02,1.552000e+03,5.000000e+00,1.605000e+03,1.554000e+03,-4.000000e+00,0.000000e+00,0.000000e+00,1.000000,0.000000,5.000000,1.000000,0.000000
75%,2015.0,9.000000e+00,2.300000e+01,6.000000e+00,3.214000e+03,3.985841e+01,-8.442694e+01,4.077724e+01,-8.131603e+01,1.822000e+03,1.832000e+03,1.100000e+01,2.000000e+01,1.846000e+03,1.660000e+02,1.630000e+02,1.370000e+02,1.017000e+03,1.957000e+03,7.000000e+00,2.006000e+03,1.959000e+03,1.000000e+01,0.000000e+00,0.000000e+00,16.000000,0.000000,23.000000,27.000000,0.000000
max,2015.0,1.200000e+01,3.100000e+01,7.000000e+00,9.793000e+03,4.197960e+01,-8.442694e+01,6.481368e+01,-6.479856e+01,2.359000e+03,2.400000e+03,1.492000e+03,2.250000e+02,2.400000e+03,6.200000e+02,6.790000e+02,6.140000e+02,4.502000e+03,2.400000e+03,2.480000e+02,2.359000e+03,2.400000e+03,1.480000e+03,1.000000e+00,1.000000e+00,862.000000,364.000000,1480.000000,1331.000000,1068.000000


In [9]:
#selecting columns that are dtype != numeric
non_numeric = df_air.select_dtypes(exclude = np.number).columns

In [10]:
#Departure Delay - response
#altered from https://www.statology.org/one-hot-encoding-in-python/

#creating instance of one-hot-encoder
encoder = OneHotEncoder(handle_unknown='ignore')

#perform one-hot encoding on 'team' column 
encoder_df = pd.DataFrame(encoder.fit_transform(df_air[non_numeric]).toarray())

#return column names from encoder
encoder_df.columns = encoder.get_feature_names_out()

#merge one-hot encoded columns back with original DataFrame
encoded = df_air.join(encoder_df)

In [ ]:
#drop original columns
final_df = encoded.drop(non_numeric, axis=1)

#head(final_df)

In [ ]:
final_df.dropna()

In [ ]:
#logistic regression code altered from https://www.datacamp.com/tutorial/understanding-logistic-regression-python

feature_cols = final_df.loc[:, final_df.columns != 'departure_delay'].columns
X = final_df[feature_cols] # Features
y = final_df.departure_delay # Target variable

In [ ]:
#split dataset in features and target variable
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.25,random_state=0)

In [ ]:
# instantiate the model (using the default parameters)
logreg = LogisticRegression()

# fit the model with data
logreg.fit(X_train,y_train)

#
y_pred=logreg.predict(X_test)